## Test Live Feed Evaluation

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort


def center_crop(frame):
    h, w, _ = frame.shape
    start = abs(h - w) // 2
    if h > w:
        return frame[start: start + w]
    return frame[:, start: start + h]


def main(filename):
    # constants
    index_to_letter = list('ABCDEFGHIKLMNOPQRSTUVWXY')
    mean = 0.485 * 255.
    std = 0.229 * 255.

    # create runnable session with exported model
    ort_session = ort.InferenceSession(filename)
    
    
    cap = cv2.VideoCapture(1)
    
    #cap.open(0, cv2.CAP_DSHOW)
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        #frame = cv2.flip(frame,1)
        # preprocess data
        frame = center_crop(frame)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        x = cv2.resize(frame, (28, 28))
        x = (x - mean) / std
        
        #importedimage = cv2.imread("/Users/elantonfernandes/Desktop/HBRS/SEMESTER_4/DLRV/SignLanguage/CollectedImages/A/A_5.jpg")
        #print(importedimage)
        x = x.reshape(1, 1, 28, 28).astype(np.float32)
        y = ort_session.run(None, {'input': x})[0]

        index = np.argmax(y, axis=1)
        letter = index_to_letter[int(index)]

        cv2.putText(frame, letter, (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 4.0, (0, 255, 0), thickness=4)
        cv2.imshow("Sign Language Translator", cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [1]:
if __name__ == '__main__':
    USE_KAGGLE_DATASET = True
    if(USE_KAGGLE_DATASET):
        main(filename="signlanguage_kaggle.onnx")
    else:
        main(filename="signlanguage_collected.onnx")